In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
os.chdir("/home/yangkai/codegen-detection/")

In [2]:
import pandas as pd
gen_dataset = pd.read_pickle("dataset/csn_gpt_gen_test.pkl")
hum_dataset = pd.read_pickle("dataset/csn_1k_hum_test.pkl")

In [3]:
from utils import load_json_dataset
gen_resample_data = load_json_dataset("prompt_gpt/csn_gpt_gen_test_coderesample_temp07.jsonl")
hum_resample_data = load_json_dataset("prompt_gpt/csn_1k_hum_test_coderesample_temp07.jsonl")

In [4]:
import re
def extract_code(resp):
    try:
        text = resp['choices'][0]['message']['content']
    except KeyError:
        return ''
    pattern = r'```python\n(.+?)\n```'
    code_blocks = re.findall(pattern, text, re.DOTALL)
    if len(code_blocks) > 0:
        return code_blocks[0]

    if resp['choices'][0]['finish_reason'] == 'length':
        pattern2 = r'```python\n(.+?)'
    else:
        pattern2 = r'``` python\n(.+?)\n```'

    # pattern2 = r'```python\n(.+?)'
    code_blocks = re.findall(pattern2, text, re.DOTALL)
    if len(code_blocks) > 0:
        return code_blocks[0]

    pattern3 = r"```\n(.+?)\n```"
    code_blocks = re.findall(pattern3, text, re.DOTALL)
    if len(code_blocks) > 0:
        return code_blocks[0]

    if resp['choices'][0]['finish_reason'] == 'length':
        pattern4 = r"```\n(.+?)"
        code_blocks = re.findall(pattern4, text, re.DOTALL)
        if len(code_blocks) > 0:
            return code_blocks[0]

    # pattern4 = r"```\n(.+?)"
    # code_blocks = re.findall(pattern4, text, re.DOTALL)
    # if len(code_blocks) > 0:
    #     return code_blocks[0]

    return ""


extracted_gen_code = []
cnt = 0
for i,resp in enumerate(gen_resample_data):
    code = extract_code(resp)
    if len(code) == 0:
        print(f"ID: {i}. No Match!")
        cnt += 1
    extracted_gen_code.append(code)

cnt

ID: 119. No Match!
ID: 127. No Match!


2

In [5]:
from prepro.code_prepro import CommentsRemover,PygmentsTokenizer,EmptyLinesRemover
tokenizer = PygmentsTokenizer('Python')
comment_remover = CommentsRemover(tokenizer)
emptyline_remover = EmptyLinesRemover()

In [6]:
gen_dataset['gpt_resampled_code'] = gen_resample_data
# hum_dataset['gpt_resampled_code2'] = hum_resample_data

gen_dataset['gpt_resampled_code'] = gen_dataset['gpt_resampled_code'].apply(extract_code)
# hum_dataset['gpt_resampled_code2'] = hum_dataset['gpt_resampled_code2'].apply(extract_code)

In [8]:
temp = gen_dataset.gpt_resampled_code.apply(comment_remover.filter)
gen_dataset['gpt_resampled_code_cleaned'] = temp.apply(emptyline_remover.filter)

# temp = hum_dataset.gpt_resampled_code2.apply(comment_remover.filter)
# hum_dataset['gpt_resampled_code_cleaned2'] = temp.apply(emptyline_remover.filter)

In [9]:
gen_dataset

,code,extracted_full_func,temp,prompt,pid,gpt_resampled_code,gpt_resampled_code_cleaned
0,import string\nfrom collections import Counter...,import string\nfrom collections import Counter...,0.7,def correct_word(word_string):\n '''\n F...,0,import re\nfrom collections import defaultdict...,import re\nfrom collections import defaultdict...
1,"def find_word_prob(word_string, word_total=sum...","def find_word_prob(word_string, word_total=sum...",0.7,"def find_word_prob(word_string, word_total=sum...",1,"def find_word_prob(word_string, word_distribut...","def find_word_prob(word_string, word_distribut..."
2,"def kmeans_clustering(self, numc, X=None, npcs...","def kmeans_clustering(self, numc, X=None, npcs...",0.7,"def kmeans_clustering(self, numc, X=None, npcs...",2,from sklearn.cluster import KMeans\nfrom sklea...,from sklearn.cluster import KMeans\nfrom sklea...
3,"import tempfile\n\ndef _tmp_html_file(self, co...","import tempfile\ndef _tmp_html_file(self, cont...",0.7,"def _tmp_html_file(\n self,\n ...",3,import tempfile\n\ndef create_tmp_html_file(co...,import tempfile\ndef create_tmp_html_file(cont...
4,"def sort_numpy(array, col=0, order_back=False)...","def sort_numpy(array, col=0, order_back=False)...",0.7,"def sort_numpy(array, col=0, order_back=False)...",4,"def sort_numpy(array, col=0, order_back=False)...","def sort_numpy(array, col=0, order_back=False)..."
...,...,...,...,...,...,...,...
249,from fuzzywuzzy import fuzz\n\ndef fuzzmatch(s...,from fuzzywuzzy import fuzz\ndef fuzzmatch(sel...,0.7,"def fuzzmatch(self, fuzzkey, multi=False):\n ...",249,from fuzzywuzzy import process\n\ndef fuzzy_ma...,from fuzzywuzzy import process\ndef fuzzy_matc...
250,import zipfile\nimport os\n\ndef extract_zipdi...,import zipfile\nimport os\ndef extract_zipdir(...,0.7,"def extract_zipdir(zip_file):\n """"""\n Ex...",250,import shutil\nimport os\n\ndef extract_zipdir...,import shutil\nimport os\ndef extract_zipdir(z...
251,"def read_field_report(path, data_flag=""*DATA"",...","def read_field_report(path, data_flag=""*DATA"",...",0.7,"def read_field_report(path, data_flag = ""*DATA...",251,"def read_field_report(path, data_flag=""*DATA"",...","def read_field_report(path, data_flag=""*DATA"",..."
252,"import html\n\ndef escape(t):\n """"""\n HT...",import html\ndef escape(t):\n return html.e...,0.7,"def escape(t):\n """"""HTML-escape the text in...",252,import cgi\n\ndef escape(t):\n return cgi.e...,import cgi\ndef escape(t):\n return cgi.esc...


In [10]:
gen_dataset.to_pickle("dataset/csn_gpt_gen_test.pkl")
# hum_dataset.to_pickle("dataset/csn_1k_hum_test.pkl")